In [55]:
import numpy as np

### Example
* Suppose we have the following optimization problem:
$$
\text{maximize} \quad x_1 + 2x_2 + 4x_3
\begin{aligned}
3x_1 + x_2 + 5x_3 &\leq 10 \\
x_1 + 4x_2 + x_3 &\leq 8 \\
2x_1 + 0x_2 + 2x_3 &\leq 7 \\
\end{aligned}
$$

* The standard form of this optimization problem is given as:
$$
\text{maximize} \quad x_1 + 2x_2 + 4x_3 + 0x_4 + 0x_ 5 + 0x_6 \equiv \text{minimize} \quad -x_1 - 2x_2 - 4x_3 - 0x_4 - 0x_ 5 - 0x_6
$$
s.t.
$$
\begin{aligned}
3x_1 + x_2 + 5x_3 + x_4 &= 10 \\
x_1 + 4x_2 + x_3 + x_5 &= 8 \\
2x_1 + 0x_2 + 2x_3 + x_6 &= 7 \\
\end{aligned}
$$

* We can express this optimization problem with the following matrix notation:
$$
\text{maximize} \quad \mathbf{d}^T\mathbf{x}
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$
with $\mathbf{d} = \begin{bmatrix} 1 \\ 2 \\ 4 \\ 0 \\ 0 \\ 0 \end{bmatrix}$, $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \end{bmatrix}$, 
$\mathbf{A} = \begin{bmatrix} 3 & 1 & 5 & 1 & 0 & 0 \\ 1 & 4 & 1 & 0 & 1 & 0 \\ 2 & 0 & 2 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0  \\ 0 & 0 & 0 & 0 & 0 & 0  \end{bmatrix}$, $\mathbf{b} = \begin{bmatrix} 10 \\ 8 \\ 7 \\ 0 \\ 0 \\ 0 \end{bmatrix}$

### Approach 1

In [24]:
def admm(A, b, d, x_0, rho):
    """
    Solves the linear programming problem:

    minimize: d^T * x
    subject to: A * x <= b

    Parameters:
    A: The coefficient matrix
    b: The right-hand side vector
    c: The objective function coefficients
    x0: The initial guess
    rho: The penalty parameter

    Returns:
    The solution to the linear programming problem
    """

    # Initialize the variables
    x = x_0
    z = np.zeros(A.shape[1])
    u = np.zeros(A.shape[0])

    # Iterate over the subproblems
    for i in range(10000):
        # Solve the first subproblem
        x = np.linalg.solve(A.T * A + rho * sp.eye(A.shape[1]), b - A.T * z + rho * u)

        # Solve the second subproblem
        z = np.maximum(0, b - A * x + rho * u / rho)

        # Update the variables and the residuals
        u = u + A * x - z
        r1 = A * x - b
        r2 = d - A.T * z

        # Check for convergence
        if np.linalg.norm(r1) < 1e-16 and np.linalg.norm(r2) < 1e-16:
            break

    return x, z, u

d = np.array([-1, -2, -4])
x = np.array([0, 0, 0])
A = np.array([[3, 1, 5], 
              [1, 4, 1], 
              [2, 0, 2]])
b = np.array([10, 8, 7])

x, z , u = admm(A, b, d, x, 0.0001)
print(x); print(z); print(u)


[[ 0.43976612  0.7891891   1.05901842]
 [ 0.55883365 -0.21428308  0.37127055]
 [ 0.54829601  0.02682657 -1.59574934]]
[[6.00844415 0.78771751 0.        ]
 [0.4406076  0.         0.47287048]
 [0.         0.         0.        ]]
[[-7.36140327e+00 -6.42162179e+00  7.57129892e-01]
 [-8.88233270e+00 -1.06674874e+01 -6.25745889e+00]
 [-3.40899028e+04 -8.00000000e+00 -2.79329184e+04]]


### Approach 2
* Given the following optimization problem:
$$
\text{minimize} \quad \mathbf{d}^T\mathbf{x}
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

* We can formulate it as:
$$
\text{minimize} \quad \frac{\rho}{2}||\mathbf{x} - \alpha||^2_2
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

with $\rho = 1$, $\mathbf{A} = \begin{bmatrix} 3. & 1. & 5. \\ 1. & 4. & 1. \\ 2. & 0. & 2. \end{bmatrix}$, $\mathbf{b} = \begin{bmatrix} 10. \\ 8. \\ 7. \end{bmatrix}$, and $\mathbf{d} = \begin{bmatrix} -1 \\ -2 \\ -4 \end{bmatrix}$

* Step 1: Construct the construction matrix $\mathbf{C} = \begin{bmatrix} \rho\mathbf{I} & \mathbf{A}^T \\ \mathbf{A} & 0  \end{bmatrix}$
* Step 2: Construct the vector $\mathbf{m} = \begin{bmatrix} \rho\pmb{\alpha} \\ \mathbf{b} \end{bmatrix}$ with $\pmb{\alpha}:= \mathbf{y}^k - (1/\rho)(\pmb{\mu}^k + \mathbf{d})$ at $k = 0$
* Step 3: Solve the equation $\mathbf{C}\begin{bmatrix}\mathbf{x} \\ \pmb{\lambda}\end{bmatrix}= \mathbf{m}$

In [54]:
d = np.array([-1., -2., -4.]).T
#A = np.array([[3., 1., 5.], [1., 4., 1.], [2., 0., 2.]])
A = np.eye(3)
b = np.array([1., 8., 7.]).T
rho = 1

##
# Construct the coefficient matrix:
#
I = np.identity(3)
zeros = np.zeros(shape = (3,3))
C = np.block([[rho * I, A.T], [A, zeros]])

MAX_ITER = 5
np.random.seed(49)
y = 0*np.random.normal(size = (1,3))
mu = 0*np.random.normal(size = (1,3))

for i in range(MAX_ITER):
    ### Solve for x
    ##
    # Construct at k = 0:
    #
    # print(d)
    # print(y)
    # print(mu)
    alpha = y - (1/rho) * (mu + d)
    # print(alpha)
    # print(b)
    m = np.block([rho * alpha, b])
    

    ##
    # Solve at k = 0: 
    #
    x = np.linalg.solve(C, m.T)[:3]

    ### Solve for y:
    beta = x + (1/rho) * mu.T
    beta[beta < 0] = 0
    y = beta.T

    ### Solve for mu:
    mu1 = rho * (x.T - y)
    mu = mu + mu1

    print(m); print(x); print(y); print(mu)


[[1. 2. 4. 1. 8. 7.]]
[[1.]
 [8.]
 [7.]]
[[1. 8. 7.]]
[[0. 0. 0.]]
[[ 2. 10. 11.  1.  8.  7.]]
[[1.]
 [8.]
 [7.]]
[[1. 8. 7.]]
[[0. 0. 0.]]
[[ 2. 10. 11.  1.  8.  7.]]
[[1.]
 [8.]
 [7.]]
[[1. 8. 7.]]
[[0. 0. 0.]]
[[ 2. 10. 11.  1.  8.  7.]]
[[1.]
 [8.]
 [7.]]
[[1. 8. 7.]]
[[0. 0. 0.]]
[[ 2. 10. 11.  1.  8.  7.]]
[[1.]
 [8.]
 [7.]]
[[1. 8. 7.]]
[[0. 0. 0.]]
